In [1]:
import os,json,sys
sys.path.append(os.path.abspath(".")) 
sys.path.append(os.path.abspath(".."))
sys.path.append(os.path.abspath("aside")) # to access aside.experiments 
from torch.utils.data import Dataset
from transformers import AutoTokenizer
import re
from tqdm import tqdm
from utils.utils import *
from datasets import load_dataset
import ast
from collections import defaultdict
from utils.model_utils import load_model
from vllm import SamplingParams
os.environ['TOKENIZERS_PARALLELISM'] = 'false'  # to suppress warning message

INFO 10-10 16:19:42 [__init__.py:216] Automatically detected platform cuda.


/home/wjyeo/anaconda3/envs/ipi_hw/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/home/wjyeo/anaconda3/envs/ipi_hw/compiler_compat/ld: warning: librt.so.1, needed by /usr/local/cuda-12.4/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/wjyeo/anaconda3/envs/ipi_hw/compiler_compat/ld: warning: libpthread.so.0, needed by /usr/local/cuda-12.4/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/wjyeo/anaconda3/envs/ipi_hw/compiler_compat/ld: warning: libstdc++.so.6, needed by /usr/local/cuda-12.4/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/wjyeo/anaconda3/envs/ipi_hw/compiler_compat/ld: warning: libm.so.6, needed by /usr/local/cuda-12.4/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/wjyeo/anaconda3/envs/ipi_hw/compiler_compat/ld: /usr/local/cuda-12.4/lib64/libcufile.so: undefined reference to `std::runtime_error::~runtime_error()@GLIBCXX_3.4'
/

In [2]:
# model_path = "Qwen/Qwen3-8B"
model_path = '/mnt/disk1/xulin/models/Qwen3-8B'
# tokenizer = AutoTokenizer.from_pretrained(model_path)
use_vllm = True
model,tokenizer,is_aside,_ = load_model(model_path,use_vllm=use_vllm,dtype=torch.bfloat16,vllm_kwargs = {'gpu_memory_utilization':0.8,'enable_chunked_prefill':True}) # load model if generating the toucan dpo

INFO 10-10 16:19:45 [utils.py:328] non-default args: {'max_model_len': 32768, 'gpu_memory_utilization': 0.8, 'disable_log_stats': True, 'enable_chunked_prefill': True, 'model': '/mnt/disk1/xulin/models/Qwen3-8B'}
INFO 10-10 16:19:50 [__init__.py:742] Resolved architecture: Qwen3ForCausalLM


`torch_dtype` is deprecated! Use `dtype` instead!


INFO 10-10 16:19:50 [__init__.py:1815] Using max model len 32768
INFO 10-10 16:19:52 [scheduler.py:222] Chunked prefill is enabled with max_num_batched_tokens=16384.
WARNING 10-10 16:19:53 [__init__.py:2974] We must use the `spawn` multiprocessing start method. Overriding VLLM_WORKER_MULTIPROC_METHOD to 'spawn'. See https://docs.vllm.ai/en/latest/usage/troubleshooting.html#python-multiprocessing for more information. Reasons: CUDA is initialized
INFO 10-10 16:19:57 [__init__.py:216] Automatically detected platform cuda.
(EngineCore_DP0 pid=1337869) INFO 10-10 16:19:58 [core.py:654] Waiting for init message from front-end.
(EngineCore_DP0 pid=1337869) INFO 10-10 16:19:58 [core.py:76] Initializing a V1 LLM engine (v0.10.2) with config: model='/mnt/disk1/xulin/models/Qwen3-8B', speculative_config=None, tokenizer='/mnt/disk1/xulin/models/Qwen3-8B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_

[W1010 16:19:59.496991273 ProcessGroupNCCL.cpp:981] Warning: TORCH_NCCL_AVOID_RECORD_STREAMS is the default now, this environment variable is thus deprecated. (function operator())


[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
(EngineCore_DP0 pid=1337869) INFO 10-10 16:19:59 [parallel_state.py:1165] rank 0 in world size 1 is assigned as DP rank 0, PP rank 0, TP rank 0, EP rank 0
(EngineCore_DP0 pid=1337869) WARNING 10-10 16:19:59 [topk_topp_sampler.py:69] FlashInfer is not available. Falling back to the PyTorch-native implementation of top-p & top-k sampling. For the best performance, please install FlashInfer.
(EngineCore_DP0 pid=1337869) INFO 10-10 16:20:00 [gpu_model_ru

Loading safetensors checkpoint shards:   0% Completed | 0/5 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  20% Completed | 1/5 [00:00<00:02,  1.34it/s]
Loading safetensors checkpoint shards:  40% Completed | 2/5 [00:01<00:02,  1.27it/s]
Loading safetensors checkpoint shards:  60% Completed | 3/5 [00:02<00:01,  1.43it/s]
Loading safetensors checkpoint shards:  80% Completed | 4/5 [00:02<00:00,  1.44it/s]
Loading safetensors checkpoint shards: 100% Completed | 5/5 [00:03<00:00,  1.82it/s]
Loading safetensors checkpoint shards: 100% Completed | 5/5 [00:03<00:00,  1.59it/s]
(EngineCore_DP0 pid=1337869) 


(EngineCore_DP0 pid=1337869) INFO 10-10 16:20:03 [default_loader.py:268] Loading weights took 3.17 seconds
(EngineCore_DP0 pid=1337869) INFO 10-10 16:20:03 [gpu_model_runner.py:2392] Model loading took 15.2683 GiB and 3.362312 seconds
(EngineCore_DP0 pid=1337869) INFO 10-10 16:20:09 [backends.py:539] Using cache directory: /home/wjyeo/.cache/vllm/torch_compile_cache/426d22dfc4/rank_0_0/backbone for vLLM's torch.compile
(EngineCore_DP0 pid=1337869) INFO 10-10 16:20:09 [backends.py:550] Dynamo bytecode transform time: 5.34 s
(EngineCore_DP0 pid=1337869) INFO 10-10 16:20:10 [backends.py:194] Cache the graph for dynamic shape for later use
(EngineCore_DP0 pid=1337869) INFO 10-10 16:20:13 [backends.py:215] Compiling a graph for dynamic shape takes 3.83 s
(EngineCore_DP0 pid=1337869) INFO 10-10 16:20:14 [monitor.py:34] torch.compile takes 9.18 s in total
(EngineCore_DP0 pid=1337869) INFO 10-10 16:20:15 [gpu_worker.py:298] Available KV cache memory: 90.83 GiB
(EngineCore_DP0 pid=1337869) INFO

Capturing CUDA graphs (mixed prefill-decode, PIECEWISE): 100%|██████████| 67/67 [00:02<00:00, 23.53it/s]


(EngineCore_DP0 pid=1337869) INFO 10-10 16:20:19 [gpu_model_runner.py:3118] Graph capturing finished in 3 secs, took 0.77 GiB
(EngineCore_DP0 pid=1337869) INFO 10-10 16:20:19 [gpu_worker.py:391] Free memory on device (139.21/139.81 GiB) on startup. Desired GPU memory utilization is (0.8, 111.85 GiB). Actual usage is 15.27 GiB for weight, 5.68 GiB for peak activation, 0.07 GiB for non-torch memory, and 0.77 GiB for CUDAGraph memory. Replace gpu_memory_utilization config with `--kv-cache-memory=96544422195` to fit into requested memory, or `--kv-cache-memory=125921838592` to fully utilize gpu memory. Current kv cache memory in use is 97532180787 bytes.
(EngineCore_DP0 pid=1337869) INFO 10-10 16:20:19 [core.py:218] init engine (profile, create kv cache, warmup model) took 15.37 seconds
INFO 10-10 16:20:19 [llm.py:295] Supported_tasks: ['generate']
INFO 10-10 16:20:19 [__init__.py:36] No IOProcessor plugins requested by the model


# Generate SFT for ASIDE

1. Alpaca (50% without input, 50% with)
2. Toucan (50% tool label- take 1st turn - unrotated, 50% 2nd turn onwards, rotated.)

In [3]:
import io

OTHER_DELM_TOKENS = {
    'mark': ['{s}', '|{s}|', '<{s}>', '[{s}]', '<|{s}|>', '[|{s}|]', '<[{s}]>', '\'\'\'{s}\'\'\'', '***{s}***'],
    'inst': ['Command', 'Rule', 'Prompt', 'Task'],
    'inpt': ['Data', 'Context', 'Text'],
    'resp': ['Output', 'Answer', 'Reply'],
    'user': ['', 'Prompter ', 'User ', 'Human '],
    'asst': ['', 'Assistant ', 'Chatbot ', 'Bot ', 'GPT ', 'AI '],
}

def create_injection_for_completion(response, instruction):
    mark = np.random.choice(OTHER_DELM_TOKENS['mark']) + ':'
    
    def sample_delm(delm_name):
        role_name = 'user' if (delm_name == 'inst' or delm_name == 'inpt') else 'asst'
        role = np.random.choice(OTHER_DELM_TOKENS[role_name]) 
        delm = np.random.choice(OTHER_DELM_TOKENS[delm_name])
        p = np.random.rand()
        if p < 1/3: return (role + delm).upper()
        elif p < 2/3: return (role + delm).lower()
        else: return role + delm
    
    text = mark.format(s=sample_delm('resp')) + '\n\n' + response 
    text += '\n\n' + mark.format(s=sample_delm('inst')) + '\n\n' + instruction
    return text

def jload(f, mode="r", num_samples=None):
    if not isinstance(f, io.IOBase): f = open(f, mode=mode)
    jdict = json.load(f)
    f.close()
    return jdict

def jdump(obj, f, mode="w", indent=4, default=str):
    if not isinstance(f, io.IOBase): f = open(f, mode=mode)
    if isinstance(obj, (dict, list)): json.dump(obj, f, indent=indent, default=default)
    elif isinstance(obj, str): f.write(obj)
    else: raise ValueError(f"Unexpected type: {type(obj)}")
    f.close()


def calculate_length_for_preference_dataset(dataset, tokenizer):
    chosen_input_ids = tokenizer([d['chosen'] for d in dataset], add_special_tokens=False)["input_ids"]
    rejected_input_ids = tokenizer([d['rejected'] for d in dataset], add_special_tokens=False)["input_ids"]

    chosen_lengths = np.array([len(prompt) for prompt in chosen_input_ids])
    rejected_lengths = np.array([len(prompt) for prompt in rejected_input_ids])
    prompt_and_label_lengths = np.maximum(chosen_lengths,rejected_lengths)

    print('Input+Output model_max_length (98%, 99%, 99.5%, 99.9%):', np.percentile(prompt_and_label_lengths, [95, 99, 99.5, 99.9]))
    print (f'Mean: {(np.mean(chosen_lengths) + np.mean(rejected_lengths))/2:.2f} Num > 2048: {np.sum(prompt_and_label_lengths>2048)} / {len(prompt_and_label_lengths)}')

In [ ]:
data_dir = "/mnt/disk1/wjyeo/data"
sft_data_path = {'train':os.path.join(data_dir, "alpaca_aside_train.json"),
                        'val':os.path.join(data_dir, "alpaca_aside_val.json"),}
clean_alpaca = load_dataset("yahma/alpaca-cleaned")['train']
print (len(clean_alpaca))

51760


In [6]:
alpaca_sampling_params = SamplingParams(temperature=0., max_tokens=2048, stop=tokenizer.eos_token)

data_with_input = []
data_without_input = []
for i,sample in enumerate(clean_alpaca):
    if sample.get('input', '').strip() == '':
        data_without_input.append(sample)
    else:
        data_with_input.append(sample)
print (len(data_with_input), len(data_without_input))

balanced_alpaca = []
num_samples = min(len(data_with_input), len(data_without_input))
if len(data_with_input) > num_samples:
    data_with_input = np.random.choice(data_with_input, num_samples, replace=False).tolist()
if len(data_without_input) > num_samples:
    data_without_input = np.random.choice(data_without_input, num_samples, replace=False).tolist()
balanced_alpaca = data_with_input + data_without_input
print (len(balanced_alpaca))
np.random.shuffle(balanced_alpaca)

19157 32603
38314


In [ ]:
sft_inputs = []
for sample in balanced_alpaca:
    if sample.get('input','').strip() != '':
        conv = [
            {"role": "user", "content": sample["instruction"] + "\n\n" + sample["input"]}
        ]
    else:
        conv = [
            {"role": "user", "content": sample["instruction"]}
        ]
    sft_inputs.append(tokenizer.apply_chat_template(conv, tokenize=False, add_generation_prompt=True,enable_thinking=False))
print (f'SFT dataset size: {len(sft_inputs)}')  
sft_outputs = model.generate(sft_inputs, alpaca_sampling_params,use_tqdm=True)
sft_outputs = [o.outputs[0].text for o in sft_outputs]

sft_data = []
for sample,output in zip(balanced_alpaca,sft_outputs):
    if sample.get('input','').strip() == '':
        sft_data.append({
            'instruction': sample['instruction'],
            'output': output
        })
    else:
        sft_data.append({
            'instruction': sample['instruction'],
            'input': sample.get('input',''),
            'output': output
        })

val_size= 500
random_ids = np.random.permutation(len(sft_data))
sft_data_train = [sft_data[i] for i in random_ids[val_size:]]
sft_data_val = [sft_data[i] for i in random_ids[:val_size]]
print (f'SFT train size: {len(sft_data_train)}, val size: {len(sft_data_val)}')

jdump(sft_data_train,sft_data_path['train'])
jdump(sft_data_val,sft_data_path['val'])

## Get length
input_len = [len(ids) for ids in tokenizer([d['instruction'] + '\n\n' + d.get('input','') for d in sft_data_train])["input_ids"]]
completion_len = [len(ids) for ids in tokenizer([d['output'] for d in sft_data_train])["input_ids"]]
overall_len = [i+c for i,c in zip(input_len,completion_len)]
print (f'Mean/Max/Min/>2048: {np.mean(overall_len):.2f} / {np.max(overall_len)} / {np.min(overall_len)} / {np.sum(np.array(overall_len)>2048)}')


# Generate onpolicy for Toucan

In [4]:
ds = load_dataset('Agent-Ark/Toucan-1.5M','SFT',split = 'train').to_list()

Resolving data files:   0%|          | 0/40 [00:00<?, ?it/s]

In [5]:
def try_json_loads(value): # THE JSON tool call labels are messy.
    """Try to parse value as JSON, otherwise return as-is."""
    if isinstance(value, str):
        try:
            parsed = json.loads(value)
            # Recursively process the parsed result as well
            return recursively_fix(parsed)
        except (json.JSONDecodeError, TypeError):
            return value
    return value

def recursively_fix(obj):
    """Recursively decode nested JSON strings inside dicts/lists."""
    if isinstance(obj, dict):
        return {k: recursively_fix(try_json_loads(v)) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [recursively_fix(try_json_loads(x)) for x in obj]
    else:
        return try_json_loads(obj)

def fix_tool_response(tool_response):
    tool_response = ast.literal_eval(tool_response)
    tool_response = recursively_fix(tool_response)
    return json.dumps(tool_response, ensure_ascii=False)


In [ ]:
## reformat the dataset
toolcall_format = "<tool_call>\n{tool}\n</tool_call>\n"

# Get the length.
def get_len(sample):
    convs = sample['conversations']
    formatted = tool_prompt_format(convs,sample['tools'],tokenizer)
    sample['len'] =len(tokenizer.encode(formatted))
    return sample

cat_ds = defaultdict(list)
invalid_cats = defaultdict(int)

for sample in tqdm(ds,total = len(ds)):
    try: # have to be able to parse both of them.
        tools = json.loads(sample['tools'])
        messages = json.loads(sample['messages'])
    except:
        invalid_cats['parsing_error'] += 1
        continue
    
    cat = sample['subset_name']
    roles = [m['role'] for m in messages]
    if not any([r == 'tool_response' for r in roles]): # no tool roles
        invalid_cats['no_tool_use'] += 1
        continue
    
    structured_msg = []
    curr_tool_call = ''
    assistant_start = False
    turns_with_response = [] # We add valid assistant turns with the option of choosing any random turn to only do SFT on that turn, the issue is that in Qwen, the no-thinking mode causes extra tokens <think> and </think> to be added only to the last turn, thus we if train on all turns, usually only the tool turns are earlier and do not have these <think> tokens, thus there will be a mismatch.
    tool_started = False # used to track if the conversation includes a tool response.
    turns_without_response = []


    for msg_id,m in enumerate(messages): # we ignore the assistant role since Qwen do not generate any text other than the MCP tools
        if m['role'] == 'tool_call':
            tool_response = fix_tool_response(m['content'])
            curr_tool_call += toolcall_format.format(tool = tool_response)
        else:
            if curr_tool_call != '': # if the next msg is not tool_call, we dump the curr_tool_call
                structured_msg.append({'role':'assistant','content':curr_tool_call.strip()})
                curr_tool_call = ''
                if tool_started:
                    turns_with_response.append(len(structured_msg)-1)
                else: # keep track until which turn it doesnt have any response.
                    turns_without_response.append(len(structured_msg)-1)

            if m['role'] == 'user':
                structured_msg.append(m)
            elif m['role'] == 'tool_response':
                tool_started = True
                structured_msg.append({'role':'tool','content':m['content']})
            elif m['role'] == 'assistant': # if is assistant, we only keep it if it is the last msg or if the next msg is not tool_call.
                if msg_id == len(messages) - 1 or messages[msg_id + 1]['role'] != 'tool_call':
                    structured_msg.append(m)
                    if tool_started:
                        turns_with_response.append(len(structured_msg)-1)
                    else:
                        turns_without_response.append(len(structured_msg)-1)

    for turn in turns_with_response:
        truncated_msg = structured_msg[:turn]
        assert truncated_msg[-1]['role'] != 'assistant', "Last message should not be assistant!"
        cat_ds['rotated'].append(
        {'conversations':truncated_msg,
         'tools':tools,})
    for turn in turns_without_response:
        truncated_msg = structured_msg[:turn]
        assert truncated_msg[-1]['role'] != 'assistant', "Last message should not be assistant!"
        cat_ds['non-rotated'].append(
        {'conversations':truncated_msg,
         'tools':tools,})
    # if np.random.rand() < 0.5: 
    #     chosen_turn = np.random.choice(turns_with_response)
    #     cat_label = 'rotated'
    # else:
    #     chosen_turn = np.random.choice(turns_without_response)
    #     cat_label = 'non-rotated'
    # structured_msg = structured_msg[:chosen_turn]
    

    

for k in invalid_cats:
    print(f"Invalid cat {k}: {invalid_cats[k]} samples")

for k in cat_ds:
    print(f"Valid cat {k}: {len(cat_ds[k])} samples")

print (f"Total samples with assistant tool calls: {len(cat_ds['tool'])} / {len(ds)}")


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 119287/119287 [00:25<00:00, 4757.74it/s]

Invalid cat no_tool_use: 40000 samples
Valid cat rotated: 259452 samples
Valid cat non-rotated: 79340 samples
Total samples with assistant tool calls: 0 / 119287


In [7]:
# Use multi process to get the lengths
cat_ds_w_len = {k:async_process(get_len,cat_ds[k],workers=32, msg="Computing input lengths") for k in cat_ds}


Token indices sequence length is longer than the specified maximum sequence length for this model (95538 > 32768). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (95538 > 32768). Running this sequence through the model will result in indexing errors
Computing input lengths: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 79340/79340 [01:06<00:00, 1199.94it/s]
Computing input lengths: 0it [00:00, ?it/s]


In [8]:
MAX_LENGTH = 4096-512
cat_ds_valid = {k:[s for s in cat_ds_w_len[k] if s['len'] <= MAX_LENGTH] for k in cat_ds_w_len}
for k in cat_ds_w_len:
    print (f"Cat {k}: before filtering {len(cat_ds_w_len[k])}, after filtering {len(cat_ds_valid[k])}")

Cat rotated: before filtering 259452, after filtering 137836
Cat non-rotated: before filtering 79340, after filtering 54100
Cat tool: before filtering 0, after filtering 0


In [ ]:
# we should balance it out, since the dataset has more non-tool samples. we take an equal amount of tool and non-tool samples.
tool_sampling_params = SamplingParams(temperature=0., max_tokens=512)
rotated_ds = cat_ds_valid['rotated']
non_rotated_ds = cat_ds_valid['non-rotated']

balanced_ds = rotated_ds + non_rotated_ds
np.random.shuffle(balanced_ds)

balanced_inputs = [tool_prompt_format(m['conversations'],m['tools'],tokenizer) for m in balanced_ds]

balanced_outputs = model.generate(balanced_inputs, tool_sampling_params,use_tqdm=True)
balanced_outputs = [o.outputs[0].text for o in balanced_outputs]


for sample,output in zip(balanced_ds,balanced_outputs):
    sample['conversations'].append({'role':'assistant','content':output})

random_ids = np.random.permutation(len(balanced_ds))
val_size = 1000
train_ids = random_ids[val_size:]
val_ids = random_ids[:val_size]

train_ds = [balanced_ds[i] for i in train_ids]
val_ds = [balanced_ds[i] for i in val_ids]

print (f"Train size: {len(train_ds)}, Val size: {len(val_ds)}")

with open('/mnt/disk1/wjyeo/data/toucan_aside_train.json','w') as f:
    json.dump(train_ds,f,indent = 4)
with open('/mnt/disk1/wjyeo/data/toucan_aside_val.json','w') as f:
    json.dump(val_ds,f,indent = 4)

Adding requests:   0%|          | 0/191936 [00:00<?, ?it/s]

Processed prompts:   0%|                                                                                      …